In [1]:
from makiflow.tools.xml_parser import XmlParser
from makiflow.layers import *
from makiflow.models import SSDModel, DetectorClassifier
from makiflow.models.ssd.ssd_model import OffsetRegression
import tensorflow as tf
import makiflow as mf
from makiflow.save_recover import Builder
mf.set_main_gpu(0)

In [ ]:
num2name = [0 for i in range(20)]

In [ ]:
decay_ssd = 0.9
decay_back = 0.997
eps = 1e-4
detector_bias=False
activation = tf.nn.relu6
boxes_old = [
                (1, 1), (1, 2), (2, 1),
                (1.26, 1.26), (1.26, 2.52), (2.52, 1.26),
                (1.59, 1.59), (1.59, 3.17), (3.17, 1.59)
            ]

boxes = [
                (1.0, 1.0), (1.0, 2.0), (2.0, 1.0),
                (1.41, 1.41), (2.83, 1.41), (1.41, 2.83)
            ]

def identity_block(x,in_f,stage_id,unit_id,num_block):                  
    prefix = 'stage' + str(stage_id) + '_unit' + str(unit_id) + '_'

    mx = BatchNormLayer(D=in_f,name=prefix + 'bn1')(x)
                                        
    mx = ActivationLayer(name=prefix + 'activation_1')(mx)

    mx = ZeroPaddingLayer(padding=[[1,1],[1,1]], name=prefix + 'zero_pad_1')(mx)

    mx = ConvLayer(kw=3,kh=3,in_f=in_f,out_f=in_f,activation=None,
                                    padding='VALID',use_bias=False,name=prefix + 'conv1')(mx)
                                                                                    
    mx = BatchNormLayer(D=in_f,name=prefix + 'bn2')(mx)
    
    mx = ActivationLayer(name=prefix + 'activation_2')(mx)

    mx = ZeroPaddingLayer(padding=[[1,1],[1,1]], name=prefix + 'zero_pad_2')(mx)

    mx = ConvLayer(kw=3,kh=3,in_f=in_f,out_f=in_f,activation=None,
                                    padding='VALID',use_bias=False,name=prefix + 'conv2')(mx)                        

    x = SumLayer(name='add' + str(num_block))([mx,x])

    return x

def conv_block(x,in_f,stage_id,unit_id,num_block,out_f=None,stride=2):
              
    prefix = 'stage' + str(stage_id) + '_unit' + str(unit_id) + '_'
    double_in_f = None
    if out_f is None:
        double_in_f = int(2*in_f)
    else:
        double_in_f = out_f

    x = BatchNormLayer(D=in_f, name=prefix + 'bn1')(x)
    x = ActivationLayer(name=prefix + 'activation_1')(x)

    mx = ZeroPaddingLayer(padding=[[1,1],[1,1]], name=prefix + 'zero_pad_1')(x)

    mx = ConvLayer(kw=3, kh=3, in_f=in_f, out_f=double_in_f, activation=None, stride=stride,
                                    padding='VALID', use_bias=False, name=prefix + 'conv1')(mx)
                                                                                
    mx = BatchNormLayer(D=double_in_f, name=prefix + 'bn2')(mx)
    mx = ActivationLayer(name=prefix + 'activation_2')(mx)

    mx = ZeroPaddingLayer(padding=[[1,1],[1,1]], name=prefix + 'zero_pad_2')(mx)
    mx = ConvLayer(kw=3, kh=3, in_f=double_in_f, out_f=double_in_f, activation=None,
                                    padding='VALID', use_bias=False, name=prefix + 'conv2')(mx)
                                                                                

    sx = ConvLayer(kw=1, kh=1, in_f=in_f, out_f=double_in_f, stride=stride,
                                    padding='VALID', activation=None, use_bias=False, name=prefix + 'sc/conv')(x)
                                                                               
    x = SumLayer(name='add' + str(num_block))([mx,sx])

    return x

def create_resnet34(batch_size, picture_size=300):

    in_x = InputLayer(input_shape=[batch_size,picture_size,picture_size,3], name='Input')

    x = BatchNormLayer(D=3, use_gamma=False, name='bn_data')(in_x)
    x = ZeroPaddingLayer(padding=[[3,3], [3,3]] ,name='zero_padding2d')(x)

    x = ConvLayer(kw=7, kh=7, in_f=3, out_f=64, stride=2, use_bias=False, activation=None, padding='VALID', name='conv0')(x)
    # size_picture = 150
    x = BatchNormLayer(D=64, name='bn0')(x)
    x = ActivationLayer(name='activation0')(x)

    x = ZeroPaddingLayer(padding=[[1,1],[1,1]], name='zero_padding2d_1')(x)

    x = MaxPoolLayer(ksize=[1,3,3,1], padding='VALID', name='max_pooling2d')(x)
    # size_picture = 75

    x = conv_block(x=x, in_f=64, stage_id=1, unit_id=1, num_block=0, out_f=64, stride=1)

    x = identity_block(x=x, in_f=64, stage_id=1, unit_id=2, num_block=1)
    x = identity_block(x=x, in_f=64, stage_id=1, unit_id=3, num_block=2)

    x = conv_block(x=x, in_f=64, stage_id=2, unit_id=1, num_block=3)
    # 38 out_f = 128

    x = identity_block(x=x, in_f=128, stage_id=2, unit_id=2, num_block=4)
    x = identity_block(x=x, in_f=128, stage_id=2, unit_id=3, num_block=5)
    x = identity_block(x=x, in_f=128, stage_id=2, unit_id=4, num_block=6)
    
    x = conv_block(x=x, in_f=128, stage_id=3, unit_id=1, num_block=7)
    # 19 out_f = 256

    x = identity_block(x=x, in_f=256, stage_id=3, unit_id=2, num_block=8)
    x = identity_block(x=x, in_f=256, stage_id=3, unit_id=3, num_block=9)
    x = identity_block(x=x, in_f=256, stage_id=3, unit_id=4, num_block=10)
    x = identity_block(x=x, in_f=256, stage_id=3, unit_id=5, num_block=11)
    x = identity_block(x=x, in_f=256, stage_id=3, unit_id=6, num_block=12)
    temp = x
    x = conv_block(x=x, in_f=256, stage_id=4, unit_id=1, num_block=13)
    # 10 out_f = 512

    x = identity_block(x=x, in_f=512, stage_id=4, unit_id=2, num_block=14)
    x = identity_block(x=x, in_f=512, stage_id=4, unit_id=3, num_block=15)

    x = BatchNormLayer(D=512, name='bn1')(x)
    x = ActivationLayer(name='relu1')(x)

    return in_x, x, temp


in_x, x, temp = create_resnet34(32,300)

# 19x19
dc0 = DetectorClassifier(
    reg_fms=temp, rkw=3, rkh=3, rin_f=256,
    class_fms=temp, ckw=3, ckh=3, cin_f=256, num_classes=len(num2name) + 1, 
    dboxes = boxes,name='dc0', use_reg_bias=detector_bias, use_class_bias=detector_bias
)


# 10x10
dc1 = DetectorClassifier(
    reg_fms=x, rkw=3, rkh=3, rin_f=512,
    class_fms=x, ckw=3, ckh=3, cin_f=512, num_classes=len(num2name) + 1, 
    dboxes = boxes,name='dc1', use_reg_bias=detector_bias, use_class_bias=detector_bias
)

x = ConvLayer(kw=1,kh=1,activation=None,use_bias=False,in_f=512,out_f=256,name='layer_19_1_Conv2d_2_1x1_256')(x)
x = BatchNormLayer(D=256,eps=eps,decay=decay_ssd,name='layer_19_1_Conv2d_2_1x1_256/BatchNorm')(x)
x = ActivationLayer(activation=activation, name='layer_19_1_Conv2d_2_1x1_256/activation_0')(x)

x = ConvLayer(kw=3,kh=3,activation=None,use_bias=False,in_f=256,out_f=256,name='layer_19_1_Conv2d_2_1x1_256_depthwise_1')(x)
x = BatchNormLayer(D=256,eps=eps,decay=decay_ssd,name='layer_19_2_Conv2d_2_3x3_s2_512_depthwise/BatchNorm')(x)
x = ActivationLayer(activation=activation, name='layer_19_2_Conv2d_2_3x3_s2_512_depthwise/activation_1')(x)

x = MaxPoolLayer(name='layer_19_2_Conv2d_2_3x3_s2_512_depthwise/max_pool')(x)

# 5x5
dc2 = DetectorClassifier(
    reg_fms=x, rkw=3, rkh=3, rin_f=256,
    class_fms=x, ckw=3, ckh=3, cin_f=256, num_classes=len(num2name) + 1, 
    dboxes = boxes, name='dc2', use_reg_bias=detector_bias, use_class_bias=detector_bias
)

x = ConvLayer(kw=1,kh=1,activation=None,use_bias=False,in_f=256,out_f=128,name='layer_19_1_Conv2d_3_1x1_128')(x)
x = BatchNormLayer(D=128,eps=eps,decay=decay_ssd,name='layer_19_1_Conv2d_3_1x1_128/BatchNorm')(x)
x = ActivationLayer(activation=activation, name='layer_19_1_Conv2d_3_1x1_128/activation_1')(x)

x = ConvLayer(kw=3,kh=3,in_f=128,out_f=128,use_bias=False,activation=None,name='layer_19_2_Conv2d_3_3x3_s2_256_depthwise_2')(x)
x = BatchNormLayer(D=128,eps=eps,decay=decay_ssd,name='layer_19_2_Conv2d_3_3x3_s2_256_depthwise/BatchNorm')(x)
x = ActivationLayer(activation=activation, name='layer_19_2_Conv2d_3_3x3_s2_256_depthwise/activation_2')(x)

x = MaxPoolLayer(name='layer_19_2_Conv2d_3_3x3_s2_256_depthwise/max_pool')(x)

# 3x3
dc3 = DetectorClassifier(
    reg_fms=x, rkw=3, rkh=3, rin_f=128,
    class_fms=x, ckw=3, ckh=3, cin_f=128, num_classes=len(num2name) + 1, 
    dboxes = boxes,name='dc3', use_reg_bias=detector_bias, use_class_bias=detector_bias
)

x = ConvLayer(kw=3,kh=3,in_f=128,out_f=128,use_bias=False,activation=None,name='layer_19_2_Conv2d_3_3x3_s2_256_depthwise_3')(x)
x = BatchNormLayer(D=128,eps=eps,decay=decay_ssd,name='layer_19_2_Conv2d_4_3x3_s2_256_depthwise/BatchNorm')(x)
x = ActivationLayer(activation=activation, name='layer_19_2_Conv2d_4_3x3_s2_256_depthwise/activation_2')(x)

x = MaxPoolLayer(name='layer_19_2_Conv2d_4_3x3_s2_128_depthwise/max_pool')(x)

# 2x2
dc4 = DetectorClassifier(
    reg_fms=x, rkw=3, rkh=3, rin_f=128,
    class_fms=x, ckw=3, ckh=3, cin_f=128, num_classes=len(num2name) + 1, 
    dboxes = boxes,name='dc4', use_reg_bias=detector_bias, use_class_bias=detector_bias
)

x = ConvLayer(kw=1,kh=1,activation=None,use_bias=False,in_f=128,out_f=64,name='layer_19_1_Conv2d_5_1x1_64')(x)
x = BatchNormLayer(D=64,eps=eps,decay=decay_ssd,name='layer_19_1_Conv2d_5_1x1_64/BatchNorm')(x)
x = ActivationLayer(activation=activation, name='layer_19_1_Conv2d_5_1x1_64/activation_1')(x)

x = ConvLayer(kw=3,kh=3,activation=None,use_bias=False,in_f=64,out_f=64,name='layer_19_1_Conv2d_5_1x1_64_depthwise_3')(x)
x = BatchNormLayer(D=64,eps=eps,decay=decay_ssd,name='layer_19_2_Conv2d_5_3x3_s2_128_depthwise/BatchNorm')(x)
x = ActivationLayer(activation=activation, name='layer_19_2_Conv2d_5_3x3_s2_128_depthwise/activation_2')(x)

# 1x1
dc5 = DetectorClassifier(
    reg_fms=x, rkw=1, rkh=1, rin_f=64,
    class_fms=x, ckw=1, ckh=1, cin_f=64, num_classes=len(num2name) + 1, 
    dboxes = [(1.0,1.0),(0.7,0.7),(0.5, 0.5),(1.0,0.5),(0.5,1.0)],name='dc5', use_reg_bias=detector_bias, use_class_bias=detector_bias

)

#model = SSDModel(dcs=[dc0,dc1, dc2, dc3, dc4,dc5],input_s=in_x,name='BIGMACHINE')

In [ ]:
model = SSDModel(dcs=[dc0,dc1, dc2, dc3, dc4,dc5],input_s=in_x,name='BIGMACHINE', offset_reg_type=OffsetRegression.RCNN_LIKE)

In [ ]:
model = Builder.ssd_from_json('scratch_det/model.json', batch_size=1)

In [ ]:
ses = tf.Session()
model.set_session(ses)

In [ ]:
model.save_architecture('resnet34_1_rcnn_style/resnet34_noBias.json')

In [ ]:
dbox = model.dboxes_xy

In [ ]:
model.dboxes_wh[-1]

In [ ]:
model.dboxes_xy[-1]

In [ ]:
bboxes_xy2wh(model.dboxes_xy)[-1]

In [ ]:
bboxes_wh2xy(model.dboxes_wh)[-1]

In [ ]:
model.dboxes_xy[-1]

In [ ]:
import numpy as np

In [ ]:
np.save('Mobilenetv2_1/dboxes.npy', dbox)

In [ ]:
from makiflow.models.ssd.ssd_utils import bboxes_wh2xy, bboxes_xy2wh